### 先决条件代码

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root=r"D:\Github\learning-repository\assets\data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root=r"D:\Github\learning-repository\assets\data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

### 超参数
-`Epochs`数 遍历数据集的次数
-批次大小 在更新参数之前通过网络传播的数据样本数
-学习率 在每个批次/epoch 更新模型参数的程度。较小的值产生较慢的学习速度，而较大的值可能导致训练期间出现不可预测的行为。


In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

### 优化循环
优化循环的每次迭代称为一个epoch，每个epoch由两个主要部分组成
训练循环：迭代训练数据集并尝试收敛到最佳参数。
验证/测试循环：迭代测试数据集以检查模型性能是否正在提高。

### 损失函数
常见的损失函数包括用于回归任务的 `nn.MSELoss`（均方误差）和用于分类的 `nn.NLLLoss`（负对数似然）。`nn.CrossEntropyLoss` 结合了 `nn.LogSoftmax` 和 `nn.NLLLoss`。

我们将模型的输出 `logits` 传递给 `nn.CrossEntropyLoss`，它将标准化 `logits` 并计算预测误差。


In [3]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

### 优化器
所有优化逻辑都封装在 `optimizer` 对象中。在这里，我们使用 SGD 优化器
我们通过注册需要训练的模型参数并传入学习率超参数来初始化优化器。

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### 完整实现


In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)#计算损失

        # Backpropagation
        loss.backward()# 反向传播
        optimizer.step()#调整模型参数
        optimizer.zero_grad()#梯度清零

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.301309  [   64/60000]
loss: 2.301266  [ 6464/60000]
loss: 2.280473  [12864/60000]
loss: 2.280265  [19264/60000]
loss: 2.278792  [25664/60000]
loss: 2.232024  [32064/60000]
loss: 2.247642  [38464/60000]
loss: 2.210382  [44864/60000]
loss: 2.202325  [51264/60000]
loss: 2.184188  [57664/60000]
Test Error: 
 Accuracy: 41.2%, Avg loss: 2.182625 

Epoch 2
-------------------------------
loss: 2.184051  [   64/60000]
loss: 2.185746  [ 6464/60000]
loss: 2.131168  [12864/60000]
loss: 2.143637  [19264/60000]
loss: 2.117091  [25664/60000]
loss: 2.045929  [32064/60000]
loss: 2.070114  [38464/60000]
loss: 1.998127  [44864/60000]
loss: 1.996141  [51264/60000]
loss: 1.928586  [57664/60000]
Test Error: 
 Accuracy: 56.9%, Avg loss: 1.938557 

Epoch 3
-------------------------------
loss: 1.965810  [   64/60000]
loss: 1.946470  [ 6464/60000]
loss: 1.833437  [12864/60000]
loss: 1.858107  [19264/60000]
loss: 1.780607  [25664/60000]
loss: 1.708422  [32064/600

## 保存和加载模型


In [9]:
import torchvision.models as models

torch.save(model.state_dict(), 'model_weights.pth')# 保存模型权重参数

In [10]:
model1 = NeuralNetwork()  #实例化新模型

In [12]:
model1.load_state_dict(torch.load('model_weights.pth',weights_only=True))# 将模型权重参数加载到新模型

<All keys matched successfully>

In [13]:
model1.eval() #请务必在推理前调用 model.eval() 方法，将 dropout 和批归一化层设置为评估模式。未能这样做将导致不一致的推理结果。

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)